In [1]:
import mlflow
import mlflow.keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

mlflow.set_tracking_uri("http://localhost:5000")

# Cargar los datos desde el archivo CSV
datos = pd.read_csv('datossaber11_original.csv')

# Lista de columnas seleccionadas
columnas_seleccionadas = ['cole_area_ubicacion', 'cole_bilingue', 'cole_calendario',
                          'cole_genero', 'estu_depto_presentacion',
                          'fami_estratovivienda', 'fami_tieneautomovil', 'fami_tienecomputador',
                          'fami_tieneinternet', 'fami_tienelavadora', 'desemp_ingles', 'punt_global']

# Crear un nuevo DataFrame con las columnas seleccionadas
data = datos[columnas_seleccionadas]

# Selecciona las características (X) y la variable objetivo (y)
X = data.drop('punt_global', axis=1)  # Excluye la variable objetivo
y = data['punt_global']

# Normalizar la variable objetivo
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

# Definir las columnas categóricas
categorical_features = ['cole_area_ubicacion', 'cole_bilingue', 'cole_calendario',
                        'cole_genero', 'estu_depto_presentacion',
                        'fami_estratovivienda', 'fami_tieneautomovil', 'fami_tienecomputador',
                        'fami_tieneinternet', 'fami_tienelavadora', 'desemp_ingles']

# Crear el transformador de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Crear el pipeline de preprocesamiento
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('scaler', StandardScaler())])

# Aplicar el pipeline a los datos de entrenamiento y prueba
X_train_scaled = pipeline.fit_transform(X_train)
X_test_scaled = pipeline.transform(X_test)

# Parámetros variables
epochs = 20
batch_size = 32

# Configuración y entrenamiento del modelo con MLflow
with mlflow.start_run() as run:
    # Crear y compilar el modelo
    model = Sequential([
        Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1)  # Capa de salida sin activación para regresión
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Registrar los parámetros en MLflow
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)
    
    # Entrenar el modelo
    history = model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    # Evaluar el modelo en el conjunto de prueba
    y_pred_scaled = model.predict(X_test_scaled).flatten()

    # Desnormalizar las predicciones
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Calcular métricas
    mse = mean_squared_error(y_test_actual, y_pred)
    mae = mean_absolute_error(y_test_actual, y_pred)
    mre = np.mean(np.abs((y_test_actual - y_pred) / y_test_actual))

    # Registrar métricas en MLflow
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("mre", mre)
    
    # Registrar el modelo en MLflow
    mlflow.keras.log_model(model, "model")

    # Imprimir el resultado de la ejecución
    print(f"Run ID: {run.info.run_id}")
    print(f"Mean Squared Error: {mse}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Relative Error: {mre}")
    


2024/05/28 17:20:47 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



MlflowException: API request to http://localhost:5000/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000013800FECD50>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [7]:
model.save('modelo_saber11.h5')